# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 15 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel15');
models = s.models;


In [5]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans BCM
BCM: SMSE 0.00072408, MSLL -4.9752, NLPD -2.0957
Processing energy_data kmeans PoE
PoE: SMSE 0.00075561, MSLL -4.9460, NLPD -2.0664


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [6]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 0.00066694, MSLL -3.0485, NLPD -0.1690


### 1.1.2 Grid q

In [56]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.00069141, MSLL -3.1134, NLPD -0.2615 - q   0.20
TERBCM: SMSE 0.00068495, MSLL -3.6674, NLPD -0.8155 - q   0.40
TERBCM: SMSE 0.00067565, MSLL -3.7341, NLPD -0.8822 - q   0.60
TERBCM: SMSE 0.00066320, MSLL -3.5529, NLPD -0.7010 - q   0.80
TERBCM: SMSE 0.00064786, MSLL -3.0995, NLPD -0.2476 - q   1.00
TERBCM: SMSE 0.00063060, MSLL -2.2278, NLPD 0.6241 - q   1.20
TERBCM: SMSE 0.00061280, MSLL -0.6350, NLPD 2.2169 - q   1.40
TERBCM: SMSE 0.00059573, MSLL 2.2542, NLPD 5.1061 - q   1.60
TERBCM: SMSE 0.00058025, MSLL 7.5242, NLPD 10.3761 - q   1.80
TERBCM: SMSE 0.00056669, MSLL 17.2378, NLPD 20.0897 - q   2.00


### 1.1.3 Find single q

In [60]:
criterion = 'TERBCM';

In [61]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.6);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.6000


In [62]:
q
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

q =  0.70592
TERBCM: SMSE 0.00066945, MSLL -3.6676, NLPD -0.8157


### 1.1.4 Find multi q

In [63]:
criterion = 'TERBCM';

In [65]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1)
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
iqs =

   0.70781
   0.71375
   0.71428
   0.71058
   0.70592
   0.71215
   0.71533
   0.71050
   0.70703
   0.70920
   0.70675
   0.70784
   0.71134
   0.70696
   0.70987

Start preprocessing...
Finish preprocessing...


In [66]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.58299
   0.72842
   0.76224
   0.72126
   0.73557
   0.68640
   0.70022
   0.73712
   0.74610
   0.73268
   0.73861
   0.70454
   0.68566
   0.69772
   0.74631

TERBCM: SMSE 0.00066493, MSLL -3.6876, NLPD -0.8357


## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [5]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 0.00079676, MSLL -4.9815, NLPD -2.1297


### 1.2.2 Grid q

In [34]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 0.00175129, MSLL -4.6209, NLPD -1.7691 - q   0.20
TEGRBCM: SMSE 0.00152720, MSLL -4.6833, NLPD -1.8314 - q   0.40
TEGRBCM: SMSE 0.00124278, MSLL -4.7808, NLPD -1.9289 - q   0.60
TEGRBCM: SMSE 0.00098498, MSLL -4.8918, NLPD -2.0399 - q   0.80
TEGRBCM: SMSE 0.00079676, MSLL -4.9815, NLPD -2.1297 - q   1.00
TEGRBCM: SMSE 0.00069952, MSLL -4.9768, NLPD -2.1250 - q   1.20
TEGRBCM: SMSE 0.00069818, MSLL -4.7279, NLPD -1.8760 - q   1.40
TEGRBCM: SMSE 0.00078116, MSLL -3.9329, NLPD -1.0810 - q   1.60
TEGRBCM: SMSE 0.00091982, MSLL -1.9861, NLPD 0.8658 - q   1.80
TEGRBCM: SMSE 0.00107753, MSLL 2.3371, NLPD 5.1889 - q   2.00


### 1.2.3 Find single q

In [70]:
criterion = 'TEGRBCM';

In [71]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=1.2);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 1.2000


In [72]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [73]:
criterion = 'TEGRBCM';

In [74]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Start preprocessing...
Finish preprocessing...


In [75]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.1655
   1.1669
   1.1662
   1.1598
   1.1603
   1.1584
   1.1643
   1.1588
   1.1591
   1.1606
   1.1666
   1.1657
   1.1638
   1.1577
   1.1676

TEGRBCM: SMSE 0.00071025, MSLL -4.9901, NLPD -2.1383


### 1.2.4 Find multi q （random q )

In [76]:
criterion = 'TEGRBCM';

In [77]:
printf('Optimizing q\r\n');
iqs = 2 * rand(opts.Ms,1)
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
iqs =

   1.01626
   1.64687
   0.62155
   1.69674
   0.14253
   1.21048
   1.50212
   1.48755
   0.47275
   0.16773
   0.71925
   1.81360
   1.60160
   1.10430
   0.51252

Start preprocessing...
Finish preprocessing...


In [78]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   1.01626
   1.64687
   0.62154
   1.66748
   0.14255
   1.20657
   1.49002
   1.47866
   0.47271
   0.16774
   0.71907
   1.70328
   1.55202
   1.10264
   0.51251

TEGRBCM: SMSE 0.00108937, MSLL -4.6006, NLPD -1.7487


## About lambda

### Find multi q

In [79]:
criterion = 'TEGRBCM';

In [80]:
printf('Optimizing q\r\n');
for ldk=-5:-2
    ld = 10^ldk
    iqs = q + 0.01 * rand(opts.Ms,1);
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=ld, init_q=iqs);
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
ld =    1.0000e-05
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 0.00070941, MSLL -4.9889, NLPD -2.1370
ld =    1.0000e-04
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 0.00071010, MSLL -4.9899, NLPD -2.1380
ld =  0.0010000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 0.00070997, MSLL -4.9912, NLPD -2.1393
ld =  0.010000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 0.00070958, MSLL -4.9897, NLPD -2.1379


In [81]:
printf('Optimizing q\r\n');
for ldk=-1:4
    ld = 10^ldk
    iqs = q + 0.01 * rand(opts.Ms,1);
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=ld, init_q=iqs);
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

Optimizing q
ld =  0.10000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 0.00070939, MSLL -4.9889, NLPD -2.1371
ld =  1
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 0.00071025, MSLL -4.9899, NLPD -2.1380
ld =  10
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 0.00071049, MSLL -4.9906, NLPD -2.1387
ld =  100
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 0.00071531, MSLL -4.9916, NLPD -2.1397
ld =  1000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 0.00076609, MSLL -4.9854, NLPD -2.1335
ld =  10000
Start preprocessing...
Finish preprocessing...
TEGRBCM: SMSE 0.00105738, MSLL -4.8545, NLPD -2.0027


printf('Optimizing q\r\n');
for ldk=5:10
    ld = 10^ldk
    iqs = q + 0.01 * rand(opts.Ms,1);
    qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=ld, init_q=iqs);
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

### Find multi q (lambda, lr test)

criterion = 'TEGRBCM';

printf('Optimizing q\r\n');
for ldk=-2:3
    ld = 10^ldk
    iqs = 1.0 + 0.01 * rand(opts.Ms,1)
    qs = mul_grad_q(models, criterion, opts, iter=100000, lr=1, lambda=ld, init_q=iqs)
    [mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
endfor

## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [45]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 0.00066974, MSLL -3.0328, NLPD -0.1532


### 1.3.2 Grid q

In [46]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 0.00072082, MSLL -3.0236, NLPD -0.1440 - q   0.20
TEGPoE: SMSE 0.00071248, MSLL -3.6055, NLPD -0.7260 - q   0.40
TEGPoE: SMSE 0.00070121, MSLL -3.6799, NLPD -0.8004 - q   0.60
TEGPoE: SMSE 0.00068683, MSLL -3.4970, NLPD -0.6174 - q   0.80
TEGPoE: SMSE 0.00066974, MSLL -3.0328, NLPD -0.1532 - q   1.00
TEGPoE: SMSE 0.00065103, MSLL -2.1377, NLPD 0.7418 - q   1.20
TEGPoE: SMSE 0.00063210, MSLL -0.5009, NLPD 2.3787 - q   1.40
TEGPoE: SMSE 0.00061418, MSLL 2.4688, NLPD 5.3484 - q   1.60
TEGPoE: SMSE 0.00059805, MSLL 7.8863, NLPD 10.7658 - q   1.80
TEGPoE: SMSE 0.00058398, MSLL 17.8733, NLPD 20.7529 - q   2.00


### 1.3.3 Find single q

In [48]:
criterion = 'TEGPoE';

In [49]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.6);

Optimizing q
Msize =

 Columns 1 through 11:

   1200   1326   1099   1483    729   1584   1499   1397   1231    983   1260

 Columns 12 through 15:

   1132   1424    432   1221

Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.6000
grad_q_norm =  3.9165
grad_q_reg = -11.171
Optimizing q: remain iter - 999, q - 0.6008
grad_q_norm =  13.564
grad_q_reg = -15.799
Optimizing q: remain iter - 998, q - 0.6019
grad_q_norm =  18.220
grad_q_reg = -17.543
Optimizing q: remain iter - 997, q - 0.6032
grad_q_norm =  21.989
grad_q_reg = -22.063
Optimizing q: remain iter - 996, q - 0.6048
grad_q_norm =  8.4863
grad_q_reg = -13.646
Optimizing q: remain iter - 995, q - 0.6057
grad_q_norm = -6.5063
grad_q_reg = -4.9835
Optimizing q: remain iter - 994, q - 0.6061
grad_q_norm = -5.7156
grad_q_reg = -7.5754
Optimizing q: remain iter - 993, q - 0.6066
grad_q_norm =  10.074
grad_q_reg = -10.622
Optimizing q: remain iter - 992, q - 0.6074
grad_q_norm =  0.63802
grad_q_re

grad_q_reg =  0.41312
Optimizing q: remain iter - 815, q - 0.6820
grad_q_norm = -2.9824
grad_q_reg = -1.5430
Optimizing q: remain iter - 814, q - 0.6822
grad_q_norm = -6.0191
grad_q_reg =  1.9201
Optimizing q: remain iter - 813, q - 0.6820
grad_q_norm =  3.5710
grad_q_reg = -6.9412
Optimizing q: remain iter - 812, q - 0.6825
grad_q_norm = -10.214
grad_q_reg =  4.5541
Optimizing q: remain iter - 811, q - 0.6822
grad_q_norm = -5.2949
grad_q_reg =  3.3424
Optimizing q: remain iter - 810, q - 0.6819
grad_q_norm = -9.9978
grad_q_reg =  7.3033
Optimizing q: remain iter - 809, q - 0.6814
grad_q_norm =  3.7932
grad_q_reg = -5.8755
Optimizing q: remain iter - 808, q - 0.6818
grad_q_norm = -2.9150
grad_q_reg = -1.4015
Optimizing q: remain iter - 807, q - 0.6819
grad_q_norm =  3.8295
grad_q_reg =  1.2216
Optimizing q: remain iter - 806, q - 0.6819
grad_q_norm =  9.5854
grad_q_reg = -12.080
Optimizing q: remain iter - 805, q - 0.6827
grad_q_norm =  5.1651
grad_q_reg = -8.9132
Optimizing q: remain 

Optimizing q: remain iter - 628, q - 0.6933
grad_q_norm = -3.1856
grad_q_reg =  0.66988
Optimizing q: remain iter - 627, q - 0.6933
grad_q_norm = -7.1466
grad_q_reg = -2.6474


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




Optimizing q: remain iter - 483, q - 0.6966
grad_q_norm = -12.453
grad_q_reg =  6.5192
Optimizing q: remain iter - 482, q - 0.6962
grad_q_norm = -2.5692
grad_q_reg = -1.5067
Optimizing q: remain iter - 481, q - 0.6963
grad_q_norm = -4.4373
grad_q_reg =  1.2285
Optimizing q: remain iter - 480, q - 0.6962
grad_q_norm =  1.5884
grad_q_reg =  0.27764
Optimizing q: remain iter - 479, q - 0.6962
grad_q_norm =  1.6599
grad_q_reg = -6.8167
Optimizing q: remain iter - 478, q - 0.6967
grad_q_norm = -11.640
grad_q_reg =  5.6524
Optimizing q: remain iter - 477, q - 0.6963
grad_q_norm = -7.4137
grad_q_reg = -1.8469
Optimizing q: remain iter - 476, q - 0.6964
grad_q_norm = -11.177
grad_q_reg =  7.1026
Optimizing q: remain iter - 475, q - 0.6959
grad_q_norm =  8.7868
grad_q_reg = -10.508
Optimizing q: remain iter - 474, q - 0.6966
grad_q_norm = -8.1061
grad_q_reg =  0.041432
Optimizing q: remain iter - 473, q - 0.6966
grad_q_norm =  5.7083
grad_q_reg = -6.7728
Optimizing q: remain iter - 472, q - 0.

grad_q_norm = -9.2334
grad_q_reg =  4.1495
Optimizing q: remain iter - 295, q - 0.7004
grad_q_norm = -1.2240
grad_q_reg = -2.0063
Optimizing q: remain iter - 294, q - 0.7005
grad_q_norm = -19.911
grad_q_reg =  4.8657
Optimizing q: remain iter - 293, q - 0.7002
grad_q_norm = -14.257
grad_q_reg =  10.902
Optimizing q: remain iter - 292, q - 0.6994
grad_q_norm = -19.334
grad_q_reg =  12.950
Optimizing q: remain iter - 291, q - 0.6985
grad_q_norm = -13.269
grad_q_reg = -4.4642
Optimizing q: remain iter - 290, q - 0.6988
grad_q_norm = -18.412
grad_q_reg = -1.8224
Optimizing q: remain iter - 289, q - 0.6989
grad_q_norm =  0.74795
grad_q_reg = -4.4977
Optimizing q: remain iter - 288, q - 0.6993
grad_q_norm =  0.83442
grad_q_reg = -4.2880
Optimizing q: remain iter - 287, q - 0.6996
grad_q_norm =  2.2075
grad_q_reg = -2.7062
Optimizing q: remain iter - 286, q - 0.6998
grad_q_norm = -9.3059
grad_q_reg =  4.2364
Optimizing q: remain iter - 285, q - 0.6994
grad_q_norm = -7.6560
grad_q_reg = -2.559

grad_q_reg = -5.3892
Optimizing q: remain iter - 108, q - 0.6993
grad_q_norm = -5.9868
grad_q_reg =  0.45972
Optimizing q: remain iter - 107, q - 0.6992
grad_q_norm =  1.2863
grad_q_reg = -7.0037
Optimizing q: remain iter - 106, q - 0.6997
grad_q_norm = -11.860
grad_q_reg =  6.1247
Optimizing q: remain iter - 105, q - 0.6993
grad_q_norm = -2.0611
grad_q_reg = -3.5793
Optimizing q: remain iter - 104, q - 0.6996
grad_q_norm = -15.647
grad_q_reg =  8.3359
Optimizing q: remain iter - 103, q - 0.6990
grad_q_norm =  6.6317
grad_q_reg = -6.9699
Optimizing q: remain iter - 102, q - 0.6995
grad_q_norm = -4.6147
grad_q_reg = -1.1959
Optimizing q: remain iter - 101, q - 0.6995
grad_q_norm = -12.291
grad_q_reg =  1.0775
Optimizing q: remain iter - 100, q - 0.6995
grad_q_norm = -2.4801
grad_q_reg =  0.38724
Optimizing q: remain iter - 99, q - 0.6994
grad_q_norm = -0.37708
grad_q_reg = -1.9632
Optimizing q: remain iter - 98, q - 0.6996
grad_q_norm = -6.8075
grad_q_reg =  0.39429
Optimizing q: remain

In [50]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [51]:
printf('Optimizing q\r\n');
iqs = q + 0.01* rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=10000, lr=1e-3, lambda=10, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing qs: remain iter - 5000
qs =

   0.70578
   0.70740
   0.71295
   0.71248
   0.70576
   0.70589
   0.71125
   0.71210
   0.71210
   0.71365
   0.70671
   0.70804
   0.70994
   0.70699
   0.70893

qs_dist =    4.7383e-05
Optimizing qs: remain iter - 4999
qs =

   0.70578
   0.70741
   0.71295
   0.71248
   0.70576
   0.70589
   0.71125
   0.71210
   0.71211
   0.71361
   0.70671
   0.70804
   0.70994
   0.70700
   0.70894

qs_dist =    6.7185e-05
Optimizing qs: remain iter - 4998
qs =

   0.70574
   0.70741
   0.71296
   0.71249
   0.70577
   0.70589
   0.71125
   0.71205
   0.71212
   0.71361
   0.70673
   0.70803
   0.70993
   0.70699
   0.70895

qs_dist =    6.7388e-05
Optimizing qs: remain iter - 4997
qs =

   0.70570
   0.70742
   0.71297
   0.71251
   0.70578
   0.70589
   0.71126
   0.71204
   0.71213
   0.71361
   0.70673
   0.70799
   0.70993
   0.70699
   0.70896

qs_dist =    6.0243e-05
Optimizing qs: remai

qs_dist =    6.1055e-05
Optimizing qs: remain iter - 4929
qs =

   0.70369
   0.70746
   0.71340

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.70265
   0.70759
   0.71372
   0.71248
   0.70636
   0.70530
   0.71091
   0.71262
   0.71247
   0.71393
   0.70729
   0.70792
   0.70941
   0.70661
   0.70949

qs_dist =    6.5645e-05
Optimizing qs: remain iter - 4890
qs =

   0.70261
   0.70760
   0.71374
   0.71248
   0.70637
   0.70530
   0.71092
   0.71262
   0.71248
   0.71393
   0.70729
   0.70792
   0.70937
   0.70660
   0.70950

qs_dist =    3.8890e-05
Optimizing qs: remain iter - 4889
qs =

   0.70258
   0.70761
   0.71374
   0.71248
   0.70637
   0.70531
   0.71093
   0.71263
   0.71249
   0.71394
   0.70729
   0.70793
   0.70938
   0.70661
   0.70951

qs_dist =    3.0434e-05
Optimizing qs: remain iter - 4888
qs =

   0.70258
   0.70762
   0.71375
   0.71249
   0.70637
   0.70532
   0.71094
   0.71263
   0.71250
   0.71394
   0.70730
   0.70793
   0.70938
   0.70662
   0.70951

qs_dist =    1.6046e-05
Optimizing qs: remain iter - 4887
qs =

   0.70258
   0.70764
   0.71375
   0.71249
   0.70637
   0.70532
   0.71094
  

   0.70651
   0.70520
   0.71063
   0.71288


...

   0.62485
   0.71545
   0.73753
   0.71814
   0.72021


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.69631
   0.70170
   0.72806

qs_dist =    3.6397e-05
Optimizing qs: remain iter - 872
qs =

   0.62246
   0.71574
   0.73864
   0.71836
   0.72094
   0.69243
   0.70258
   0.72489
   0.72961
   0.72441
   0.72367
   0.70581
   0.69631
   0.70168
   0.72805

qs_dist =    3.5621e-05
Optimizing qs: remain iter - 871
qs =

   0.62245
   0.71575
   0.73864
   0.71837
   0.72095
   0.69244
   0.70259
   0.72490
   0.72962
   0.72441
   0.72368
   0.70582
   0.69632
   0.70169
   0.72806

qs_dist =    2.7544e-05
Optimizing qs: remain iter - 870
qs =

   0.62243
   0.71576
   0.73864
   0.71837
   0.72095
   0.69244
   0.70259
   0.72490
   0.72963
   0.72962
   0.72455
   0.72384
   0.70600
   0.69624
   0.70158
   0.72830

qs_dist =    1.4587e-05
Optimizing qs: remain iter - 818
qs =

   0.62181
   0.71589
   0.73887
   0.71843
   0.72118
   0.69240
   0.70248
   0.72514
   0.72963
   0.72455
   0.72384
   0.70600
   0.69624
   0.70159
   0.72831

qs_dist =    6.7948e-05
Optimizing qs:

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




qs_dist =    6.2264e-05
Optimizing qs: remain iter - 333
qs =

   0.61596
   0.71666
   0.74133
   0.71891
   0.72286
   0.69122
   0.70242
   0.72634
   0.73165
   0.72572
   0.72563
   0.70564
   0.69509
   0.70083
   0.73014

qs_dist =    3.2709e-05
Optimizing qs: remain iter - 332
qs =

   0.61595
   0.71665
   0.74134
   0.71892
   0.72286
   0.69121
   0.70241
   0.72634
   0.73166
   0.72572
   0.72564
   0.70565
   0.69510
   0.70083
   0.72282
   0.69116
   0.70245
   0.72649
   0.73170
   0.72587
   0.72585
   0.70567
   0.69499
   0.70080
   0.73043

qs_dist =    2.3942e-05
Optimizing qs: remain iter - 275
qs =

   0.61534
   0.71668
   0.74163
   0.71909
   0.72282
   0.69116
   0.70246
   0.72650
   0.73171
   0.72588
   0.72585
   0.70567
   0.69499
   0.70081
   0.73043

qs_dist =    3.6798e-05
Optimizing qs: remain iter - 274
qs =

   0.61532
   0.71670
   0.74164
   0.71909
   0.72283
   0.69117
   0.70247
   0.72651
   0.73172
   0.72589
   0.72585
   0.70568
   0.69

In [52]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.61212
   0.71704
   0.74317
   0.71944
   0.72377
   0.69060
   0.70174
   0.72750
   0.73278
   0.72654
   0.72704
   0.70546
   0.69432
   0.70025
   0.73150

TEGPoE: SMSE 0.00068981, MSLL -3.6357, NLPD -0.7561
